<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/HF_LOCAL_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu May 23 11:41:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   45C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from google.colab import drive

drive.mount('/content/gdrive',readonly=False,force_remount=True)

Mounted at /content/gdrive


In [ ]:
%mkdir -p /content/gdrive/MyDrive/model
%cd /content/gdrive/MyDrive/model

In [ ]:
!git lfs install

In [ ]:
#DONE
#!git clone https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

Cloning into 'Mistral-7B-Instruct-v0.1'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 98 (delta 48), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (98/98), 477.06 KiB | 282.00 KiB/s, done.
Filtering content: 100% (5/5), 3.46 GiB | 1.87 MiB/s, done.
fatal: cannot exec '/content/gdrive/MyDrive/model/Mistral-7B-Instruct-v0.1/.git/hooks/post-checkout': Permission denied
Encountered 4 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00002-of-00002.bin
	model-00002-of-00002.safetensors
	model-00001-of-00002.safetensors
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


In [ ]:
## firts
#!git clone https://huggingface.co/abacusai/Smaug-72B-v0.1"

### second
#!git clone https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1

#152334H/miqu-1-70b-sf

In [2]:
!pip install bitsandbytes -q
!pip install accelerate -q
!pip install sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.5 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
#from transformers import FalconForCausalLM
from transformers import pipeline

In [5]:
# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

from transformers import AutoConfig

In [6]:
from transformers import AutoModelForCausalLM

model_id = '/content/gdrive/MyDrive/model/Mistral-7B-Instruct-v0.1'

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    #attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [10]:
prompt="What was the first album Beyoncé released as a solo artist?"
#prompt="What is the capital of Russia?"

prompt="Can you explain the concepts of Quantum Computing?"
prompt = f"Instruct: Answer the following question.\n{prompt}"

#prompt = f"Instruct: Answer the following question.\n{prompt}\nOutput:\n" # for dataset_squad2

In [20]:
def get_answer(prompt):
  outputs = pipe(prompt, max_new_tokens=512, temperature=0.8, do_sample=True, top_k=50, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)
  return outputs

In [35]:
%%time
prompt = "I bought a computer for $900, sold it for $1200, repurchased it for $1300, and sold it again for $1600. how much did I earn? Take in consideration the money for the repurchased too."
outputs = get_answer(prompt)

CPU times: user 39 s, sys: 0 ns, total: 39 s
Wall time: 39 s


In [36]:
print('Question: %s'%prompt)
print()
#print('Answer: %s \nOutput:\n'%outputs[0]['generated_text'])
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Question: I bought a computer for $900, sold it for $1200, repurchased it for $1300, and sold it again for $1600. how much did I earn? Take in consideration the money for the repurchased too.

Generated Answer:
Comment: You earned $500. That's $1600 minus $1200.

Comment: @JohnWoo: or $1600-1300.

## Answer (7)

You earned $300.

Here's how to calculate that:

1. First, you bought the computer for $900.
2. Next you sold it for $1200.
3. Finally you bought it back for $1300, so you made a net loss of $300.

Your question asks how much you earned.

Comment: Yes, I'm not sure if I should have worded it differently, but in my mind it was not lost, but rather a mistake on my end.

## Answer (2)

You made $500 net profit.

Here's how to calculate that:

1. First you bought the computer for $900.
2. Next you sold it for $1200 (earning $300).
3. Then you bought it back for $1300 (incurring a loss of $300).
4. Adding all of the above shows you made a net profit of $500 ($300 earned - $300 loss)

In [32]:
%%time
prompt = "I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering."
outputs = get_answer(prompt)

CPU times: user 6.32 s, sys: 0 ns, total: 6.32 s
Wall time: 6.31 s


In [33]:
print('Question: %s'%prompt)
print()
#print('Answer: %s \nOutput:\n'%outputs[0]['generated_text'])
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Question: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.

Generated Answer:
## Answer (1)

When you bought each ice cream cone, you paid $1.25 per cone. The total cost of the 6 ice creams was $7.50.

To answer your question, you paid $10 and you got back $7.50.

Therefore, the answer is $2.50.
